In [0]:
#write json file from notebook #2 to List

import json

with open('/dbfs/mnt/<JSON file created in notebook #2>') as jsfl:
  rewards_payload = json.load(jsfl)
  jsfl.close()

In [0]:
#This cell sends a GET request for bearer token used for Rewards API

import requests

#replace <Tenant-ID> with your Azure TenantID
auth_url = "https://login.microsoftonline.com/<Tenant-ID>/oauth2/v2.0/token"

#payload and headers can be copied from sample GET request from ClientCredentialsFlow in Postman. Replace in payload  your clientID and client secret
payload='client_id=<client-id>&client_secret=<client-secret>&grant_type=client_credentials&scope=api%3A%2F%2Fcontoso-cs-rewards-api%2F.default'

headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  }

response = requests.request("GET", auth_url, headers=headers, data=payload).json()

token = json.dumps("Bearer "+response['access_token'])



In [0]:
#URL of Rewards API deployed in API-M. Replace with your APIM hostname
post_url = "https://<APIM-Hostname>.net/rewards/api/Points"

#store Subscription Key in Azure Key Vault and reference through Databricks Secret Scope, replace with your scope and key names
header = {'Authorization': token.strip('""'),
          'Ocp-Apim-Subscription-Key': dbutils.secrets.get(scope = "<Databricks Secret Scope Name>", key = "<Secret Name in Key Vault>"),
          'Content-Type': 'application/json'}

#iterate through items in List and pass each item individually to API
for item in rewards_payload:
  r = requests.post(post_url, headers = header, json = item)
  print(r)